# Data Augmented Question Answering
https://python.langchain.com/en/latest/use_cases/evaluation/data_augmented_question_answering.html 

This notebook uses some generic prompts/language models to evaluate an question answering system that uses other sources of data besides what is in the model. For example, this can be used to evaluate a question answering system over your proprietary data.

## Setup
Let's set up for my own use case.

In [1]:
%pip install -qU supabase

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.llms import LlamaCpp
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from supabase.client import Client, create_client
from langchain.vectorstores import SupabaseVectorStore
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env_supabase")

True

In [3]:
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

def get_retriever(k: int = 3):
    embedding_model_name = os.environ.get("EMBEDDING_MODEL_NAME")
    embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
    vector_store = SupabaseVectorStore(
        client=supabase, embedding=embedding, table_name="documents")
    retriever = vector_store.as_retriever(search_kwargs={"k": k})
    return retriever

print('loading model...')

llm = LlamaCpp(
        model_path="./open-llama/ggml-model-q4_0.bin",
        temperature=0.0,
        n_ctx=2048,
        verbose=True
)

loading model...


llama.cpp: loading model from ./open-llama/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 1026.00 MB per state)
llama_init_from_file: kv self size  = 1024.00 MB
AVX = 1 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [4]:
qa = RetrievalQA.from_llm(llm=llm, retriever=get_retriever())

/home/limcheekin/ws/py/flutter-gpt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-10 16:14:17,882:INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2023-05-10 16:14:21,604:INFO - Use pytorch device: cpu


## Examples
Now we need some examples to evaluate. We can do this in two ways:

1. Hard code some examples ourselves
2. Generate examples automatically, using a language model

In [5]:
# Hard-coded examples
examples = [
{
"query": "What is Flutter?",
"answer": "Flutter is an open-source mobile application development framework developed by Google. It allows developers to build high-performance, natively compiled mobile apps for iOS and Android, as well as for the web and desktop."
},
{
"query": "What programming language is used in Flutter?",
"answer": "Flutter uses Dart programming language, which is also developed by Google. Dart is a modern, object-oriented language with features like garbage collection, type inference, and asynchronous programming."
},
{
"query": "How do I install Flutter?",
"answer": "To install Flutter, you need to first download the Flutter SDK from the official Flutter website. Once downloaded, extract the contents of the zip file to a desired location on your system, and then add the Flutter SDK's bin directory to your system's PATH environment variable."
},
{
"query": "How do I create a new Flutter project?",
"answer": "To create a new Flutter project, you can use the flutter create command in the terminal, followed by the name of your project. This will create a new Flutter project with the required directory structure and files."
},
{
"query": "What is a widget in Flutter?",
"answer": "In Flutter, everything is a widget. A widget is a basic building block of a Flutter app, which can be thought of as a visual element or a part of the user interface. Widgets can be either stateful or stateless."
},
{
"query": "What is the difference between stateful and stateless widgets?",
"answer": "A stateful widget is a widget that contains mutable state, i.e., data that can change over time. A stateless widget, on the other hand, is a widget that does not contain any mutable state and is purely based on its input parameters."
},
{
"query": "How do I handle user input in Flutter?",
"answer": "To handle user input in Flutter, you can use various event handlers like onPressed for buttons, onChanged for text fields, etc. You can also use Flutter's GestureDetector widget to detect gestures like tapping, swiping, etc."
},
{
"query": "How do I navigate between screens in Flutter?",
"answer": "To navigate between screens in Flutter, you can use the Navigator class. You can push a new screen onto the navigation stack using the Navigator.push method, and pop the current screen using the Navigator.pop method."
},
{
"query": "How do I handle async operations in Flutter?",
"answer": "In Flutter, you can use the async and await keywords to handle async operations. You can use the Future class to represent a value or error that may be available at some point in the future, and use async functions to wait for these values."
},
{
"query": "What is a FutureBuilder in Flutter?",
"answer": "A FutureBuilder is a widget in Flutter that makes it easy to build UIs that depend on asynchronous data. It takes a Future as input and rebuilds itself whenever the future completes with either a value or an error."
},
{
"query": "How do I add animations in Flutter?",
"answer": "In Flutter, you can use the Animation class to create animations. You can define an animation by specifying the start and end values, a duration, and a curve. You can then use an AnimationController to control the animation and update the UI accordingly."
},
{
"query": "How do I add images in Flutter?",
"answer": "To add images in Flutter, you can use the Image widget. You can specify the source of the image using a URL or a local file path. You can also specify the width and height of the image, as well as various other properties like the fit, alignment, and color."
},
{
"query": "How do I add custom fonts in Flutter?",
"answer": "To add custom fonts in Flutter, you need to first add the font files to your project's assets directory. You then need to specify the font family and file name in your pubspec.yaml file. You can then use the TextStyle widget to apply the custom font to your text."
},
{
"query": "How do I add a splash screen in Flutter?",
"answer": "To add a splash screen in Flutter, you can create a new widget that displays your app's logo or branding, and then use it as the first screen in your app. You can then use a FutureBuilder to load the data required for your app's home screen, and navigate to it once the data is loaded."
},
{
"query": "How do I add internationalization (i18n) support to my Flutter app?",
"answer": "To add internationalization support to your Flutter app, you can use the intl package. You can define a set of messages for each supported language, and use the Localizations widget to load the appropriate message set based on the user's device language."
},
{
"query": "How do I use the camera in Flutter?",
"answer": "To use the camera in Flutter, you can use the camera package. You can use the CameraController class to control the camera and capture images or videos. You can also use various other packages for advanced camera features like barcode scanning and face detection."
},
{
"query": "How do I use the device's sensors in Flutter?",
"answer": "To use the device's sensors in Flutter, you can use various packages like sensors, flutter_blue, etc. These packages provide APIs to access the device's sensors like accelerometer, gyroscope, magnetometer, etc."
},
{
"query": "How do I use Firebase in Flutter?",
"answer": "To use Firebase in Flutter, you need to first add the Firebase SDK to your app by following the setup instructions provided by Firebase. Once done, you can use various Firebase services like authentication, database, storage, etc. using the respective Flutter plugins."
},
{
"query": "How do I test my Flutter app?",
"answer": "To test your Flutter app, you can use Flutter's built-in testing framework called flutter_test. You can write unit tests, widget tests, and integration tests using this framework. You can also use various third-party testing tools like mockito, flutter_driver, etc."
},
{
"query": "How do I deploy my Flutter app?",
"answer": "To deploy your Flutter app, you can use various methods like publishing to the app stores, creating APKs or IPA files for distribution, deploying to web or desktop platforms, etc. You can also use various third-party tools like Google Play Console, Apple App Store Connect, etc. for app store deployment."
}
]

In [6]:
examples[:10]

[{'query': 'What is Flutter?',
  'answer': 'Flutter is an open-source mobile application development framework developed by Google. It allows developers to build high-performance, natively compiled mobile apps for iOS and Android, as well as for the web and desktop.'},
 {'query': 'What programming language is used in Flutter?',
  'answer': 'Flutter uses Dart programming language, which is also developed by Google. Dart is a modern, object-oriented language with features like garbage collection, type inference, and asynchronous programming.'},
 {'query': 'How do I install Flutter?',
  'answer': "To install Flutter, you need to first download the Flutter SDK from the official Flutter website. Once downloaded, extract the contents of the zip file to a desired location on your system, and then add the Flutter SDK's bin directory to your system's PATH environment variable."},
 {'query': 'How do I create a new Flutter project?',
  'answer': 'To create a new Flutter project, you can use the f

In [ ]:
# Generated examples
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(OpenAI())

In [ ]:
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in texts[:5]])

In [ ]:
new_examples

In [ ]:
# Combine examples
examples += new_examples

## Evaluate
Now that we have examples, we can use the question answering evaluator to evaluate our question answering chain.

In [7]:
from langchain.evaluation.qa import QAEvalChain

In [8]:
predictions = qa.apply(examples[:10])

2023-05-10 16:20:48,115:WARNING - Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa05cd6d900>: Failed to establish a new connection: [Errno 111] Connection refused'))
Batches: 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]
2023-05-10 16:41:54,628:WARNING - Failed to persist run: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chain-runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa05cd51db0>: Failed to establish a new connection: [Errno 111] Connection refused'))
2023-05-10 16:41:54,632:WARNING - Failed to load default session, using empty session: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /sessions?name=default (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa05cd5

In [13]:
predictions[0]

{'query': 'What is Flutter?',
 'answer': 'Flutter is an open-source mobile application development framework developed by Google. It allows developers to build high-performance, natively compiled mobile apps for iOS and Android, as well as for the web and desktop.',
 'result': ' Flutter is a cross-platform UI toolkit that is designed to allow code reuse across operating systems such as iOS and Android, while also allowing applications to interface directly with underlying platform services. The goal is to enable developers to deliver high-performance apps that feel natural on different platforms, embracing differences where they exist while sharing as much code as possible.\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [14]:
for i, p in enumerate(predictions):
    print(f"{i}) {p['query']}")
    print(f"{p['answer']}")
    print(f"{p['result']}\n")

0) What is Flutter?
Flutter is an open-source mobile application development framework developed by Google. It allows developers to build high-performance, natively compiled mobile apps for iOS and Android, as well as for the web and desktop.
 Flutter is a cross-platform UI toolkit that is designed to allow code reuse across operating systems such as iOS and Android, while also allowing applications to interface directly with underlying platform services. The goal is to enable developers to deliver high-performance apps that feel natural on different platforms, embracing differences where they exist while sharing as much code as possible.\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [ ]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

## Evaluate with Other Metrics

In addition to predicting whether the answer is correct or incorrect using a language model, we can also use other metrics to get a more nuanced view on the quality of the answers. To do so, we can use the [Critique](https://docs.inspiredco.ai/critique/) library, which allows for simple calculation of various metrics over generated text.

First you can get an API key from the [Inspired Cognition Dashboard](https://dashboard.inspiredco.ai) and do some setup:

```bash
export INSPIREDCO_API_KEY="..."
pip install inspiredco
```

In [ ]:
import inspiredco.critique
import os
critique = inspiredco.critique.Critique(api_key=os.environ['INSPIREDCO_API_KEY'])

Then run the following code to set up the configuration and calculate the [ROUGE](https://docs.inspiredco.ai/critique/metric_rouge.html), [chrf](https://docs.inspiredco.ai/critique/metric_chrf.html), [BERTScore](https://docs.inspiredco.ai/critique/metric_bert_score.html), and [UniEval](https://docs.inspiredco.ai/critique/metric_uni_eval.html) (you can choose [other metrics](https://docs.inspiredco.ai/critique/metrics.html) too):

In [ ]:
metrics = {
    "rouge": {
        "metric": "rouge",
        "config": {"variety": "rouge_l"},
    },
    "chrf": {
        "metric": "chrf",
        "config": {},
    },
    "bert_score": {
        "metric": "bert_score",
        "config": {"model": "bert-base-uncased"},
    },
    "uni_eval": {
        "metric": "uni_eval",
        "config": {"task": "summarization", "evaluation_aspect": "relevance"},
    },
}

In [ ]:
critique_data = [
    {"target": pred['result'], "references": [pred['answer']]} for pred in predictions
]
eval_results = {
    k: critique.evaluate(dataset=critique_data, metric=v["metric"], config=v["config"])
    for k, v in metrics.items()
}

Finally, we can print out the results. We can see that overall the scores are higher when the output is semantically correct, and also when the output closely matches with the gold-standard answer.

In [ ]:
for i, eg in enumerate(examples):
    score_string = ", ".join([f"{k}={v['examples'][i]['value']:.4f}" for k, v in eval_results.items()])
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Scores: " + score_string)
    print()